# install librarys

In [4]:
! pip install transformers
! pip install tensorflow_addons
#! pip install git+https://www.github.com/keras-team/keras-contrib.git


In [5]:
import tensorflow as tf
import pandas as pd
import numpy as np
import os

from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification
from transformers import TFBertModel,  BertConfig, BertTokenizerFast
from sklearn.preprocessing import MultiLabelBinarizer
from itertools import repeat
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

from transformers import TFBertModel,  BertConfig, BertTokenizerFast
# Then what you need from tensorflow.keras
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Embedding, Bidirectional
from tensorflow.keras.layers import Input, Dropout, Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.initializers import TruncatedNormal
from tensorflow.keras.losses import CategoricalCrossentropy,BinaryCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy,BinaryAccuracy
from tensorflow.keras.utils import to_categorical
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
# And pandas for data import + sklearn because you allways need sklearn
import pandas as pd
from sklearn.model_selection import train_test_split

#from keras_contrib.layers import CRF

# Preprocessing text

In [8]:
data = pd .read_csv('/content/drive/MyDrive/NER Dataset/processed_ner_data_5.csv')

ner =  pd.DataFrame({
    'sentences' : data['sentences'],
    'tag_str' : data['tag_str']
})

labels_dict = {
'O'     : 0,    
'B-art' :   1,  
'B-eve' :   2,
'B-geo' : 3,
'B-gpe' : 4,
'B-nat' :   5,
'B-org' : 6,
'B-per' : 7,
'B-tim' : 8,
'I-art' :   9,
'I-eve' :   10,
'I-geo' :  11,
'I-gpe' :   12,
'I-nat' :    13,
'I-org' :  14,
'I-per' : 15,
'I-tim' :  16
}

ner_tag_list = list(map(lambda x: x.split(',')[1:], ner['tag_str'] ))
ner['ner_tag_list'] = ner_tag_list

In [9]:
ner_tag_list_max_len =list(map(lambda x :  ['O']+x + ( ['O']*(127-len(x))),ner['ner_tag_list']))
ner['ner_tag_list_max_len_with_cls'] = ner_tag_list_max_len

def tag_to_var(x):
  return list(map(lambda k : labels_dict[k],x))

ner_tag_list_max_len_with_cls_to_var = list(map(lambda x : tag_to_var(x), ner['ner_tag_list_max_len_with_cls'] ))
ner['ner_tag_list_max_len_with_cls_to_var'] = ner_tag_list_max_len_with_cls_to_var

ner.head()

,sentences,tag_str,ner_tag_list,ner_tag_list_max_len_with_cls,ner_tag_list_max_len_with_cls_to_var
0,Thousands of demonstrators have marched throu...,",O,O,O,O,O,O,B-geo,O,O,O,O,O,B-geo,O,O,O,O,O,B...","[O, O, O, O, O, O, B-geo, O, O, O, O, O, B-geo...","[O, O, O, O, O, O, O, B-geo, O, O, O, O, O, B-...","[0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 3, 0, ..."
1,Families of soldiers killed in the conflict j...,",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,O,B-per,O,O...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,They marched from the Houses of Parliament to...,",O,O,O,O,O,O,O,O,O,O,O,B-geo,I-geo,O","[O, O, O, O, O, O, O, O, O, O, O, B-geo, I-geo...","[O, O, O, O, O, O, O, O, O, O, O, O, B-geo, I-...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 11, 0,..."
3,"Police put the number of marchers at 10,000 w...",",O,O,O,O,O,O,O,O,O,O,O,O,O,O,O","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O]","[O, O, O, O, O, O, O, O, O, O, O, O, O, O, O, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,The protest comes on the eve of the annual co...,",O,O,O,O,O,O,O,O,O,O,O,B-geo,O,O,B-org,I-org,O...","[O, O, O, O, O, O, O, O, O, O, O, B-geo, O, O,...","[O, O, O, O, O, O, O, O, O, O, O, O, B-geo, O,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, ..."


# Modelling

In [10]:
from transformers import AutoTokenizer, AutoModel,TFAutoModel
tokenizer = AutoTokenizer.from_pretrained("ugaray96/biobert_ncbi_disease_ner")
transformer_model = TFAutoModel.from_pretrained("ugaray96/biobert_ncbi_disease_ner")
max_length = 128

Some layers from the model checkpoint at ugaray96/biobert_ncbi_disease_ner were not used when initializing TFBertModel: ['dropout_37', 'classifier']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFBertModel were not initialized from the model checkpoint at ugaray96/biobert_ncbi_disease_ner and are newly initialized: ['bert/pooler/dense/bias:0', 'bert/pooler/dense/kernel:0']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
# Ready output data for the model
y = np.array(list(ner['ner_tag_list_max_len_with_cls_to_var']),dtype=float)
# Tokenize the input (takes some time)
x = tokenizer(
    text=ner['sentences'].to_list(),
    add_special_tokens=True,
    max_length=max_length,
    truncation=True,
    padding='max_length', 
    return_tensors='tf',
    return_token_type_ids = True,
    return_attention_mask = True,
    verbose = True)

In [12]:
# Load the MainLayer
bert = transformer_model.layers[0]
# Build your model input
input_ids = Input(shape=(max_length,), dtype='int32')
token_type_ids = Input(shape=(max_length,), dtype='int32')
attention_mask = Input(shape=(max_length,), dtype='int32')


inputs = {'input_ids': input_ids,'token_type_ids':token_type_ids,'attention_mask':attention_mask}
# Load the Transformers BERT model as a layer in a Keras model
bert_model = bert(inputs)[0]
#dropout = Dropout(config.hidden_dropout_prob, name='pooled_output')
pooled_output = Dropout(0.1)(bert_model, training=False)


pooled_output = Bidirectional(LSTM(units = 50, return_sequences=True, recurrent_dropout=0.1))(pooled_output)
pooled_output = TimeDistributed(Dense(17))(pooled_output)

# And combine it all in a model object
model = Model(inputs=inputs, outputs=pooled_output, name='BERT_ner')
# Take a look at the model
model.summary()

Model: "BERT_ner"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_1 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, 128)]        0                                            
__________________________________________________________________________________________________
bert (TFBertMainLayer)          TFBaseModelOutputWit 108310272   input_3[0][0]                    
                                                                 input_1[0][0]             

In [13]:
model.compile(optimizer="adam", loss="sparse_categorical_crossentropy", metrics=["accuracy"])  

In [ ]:
history=model.fit(x={'input_ids': x['input_ids'],'token_type_ids':x['token_type_ids'],'attention_mask':x['attention_mask']},y=y,
                  #validation_data=({'input_ids': x_test['input_ids'],'token_type_ids':x_test['token_type_ids'],'attention_mask':x_test['attention_mask']},y_test),
                  batch_size=4,epochs=1)

 1910/11990 [===>..........................] - ETA: 3:26:20 - loss: 3.0478 - accuracy: 0.0038